# 177312 - Tales Lelo da Aparecida

Use os dados do arquivo *abalone*. Faça os pré-processamentos do exercício 3.

In [1]:
import math
import pandas as pd
from sklearn.preprocessing   import scale
from sklearn.model_selection import StratifiedKFold, GridSearchCV
from sklearn.neighbors       import KNeighborsClassifier
from sklearn.decomposition   import PCA
from sklearn.svm             import SVC
from sklearn.neural_network  import MLPClassifier
from sklearn.ensemble        import RandomForestClassifier, GradientBoostingClassifier

# Carrega um DataFrame com os dados do arquivo
df = pd.io.parsers.read_csv('abalone.csv', ',', header=None )

# Converte a 1ª coluna para um dado numérico
df = pd.get_dummies(df, columns=[0])

# Cria um DataFrame com a versão categórica (1:x>13, 0:x<=13) da coluna 8
greaterThan13y = df.apply(lambda row: int(row[8]>13), 1)
# Remove-a
del df[8]

# Estandardiza os dados
x = scale(df.values)
y = greaterThan13y.values

# Mostra as primeiras linhas de nossos dados, para um preview de como estão até então
df = pd.DataFrame(data=x[:5], columns=['1','2','3','4','5','6','7','F','I','M'])
df['>13y'] = greaterThan13y
pd.options.display.float_format = '{:,.3f}'.format
print("Primeiras 5 linhas:\n", df[:5])

Primeiras 5 linhas:
        1      2      3      4      5      6      7      F      I      M  >13y
0 -0.575 -0.432 -1.064 -0.642 -0.608 -0.726 -0.638 -0.675 -0.688  1.317     1
1 -1.449 -1.440 -1.184 -1.230 -1.171 -1.205 -1.213 -0.675 -0.688  1.317     0
2  0.050  0.122 -0.108 -0.309 -0.463 -0.357 -0.207  1.482 -0.688 -0.759     0
3 -0.699 -0.432 -0.347 -0.638 -0.648 -0.608 -0.602 -0.675 -0.688  1.317     0
4 -1.616 -1.541 -1.423 -1.272 -1.216 -1.287 -1.321 -0.675  1.453 -0.759     0


$\qquad$Usando um *5-fold* externo para calcular a acurácia, e um *3-fold* interno para a escolha dos hiperparâmetros, determine qual algoritmo entre **kNN**, **SVM** com kernel RBF, **redes neurais**, **Random Forest**, e **Gradient Boosting Machine** tem a maior acurácia, considerando 3 dígitos de precisão.  
$\qquad$Você não precisa fazer os loops da validação cruzada explicitamente. Pode usar a função *GridSearchCV* do SKlearn.

In [2]:
# Retorna duas listas com os índices gerados pelo kfold estratificado
def getStratifiedKFoldArrays(k, x, y):
    indexes_pairs = list(StratifiedKFold(k).split(x, y))
    train_indexes, test_indexes = [], []
    for i in range(k):
        train_indexes.append(indexes_pairs[i][0])
        test_indexes.append(indexes_pairs[i][1])
    return train_indexes, test_indexes

# Gera os índices dos 5 folds (kfold estratificado)
tr_5kf_idxs, ts_5kf_idxs = getStratifiedKFoldArrays(5, x, y)

Para o **kNN**, faça um *PCA* que mantém 90% da variância. Busque os valores do *k* entre os valores $1, 5, 11, 15, 21, 25$.

In [3]:
# Conjunto de hiperparâmetros testados para otimização do kNN
ks = [1, 5, 11, 15, 21, 25]

# Definição do objeto PCA que busca manter variância de 90%
pca = PCA(n_components=0.9)
x_reduced = pca.fit_transform(x)

print("Teste do kNN")
print(len(x[0]), "dimensões reduzidas para", pca.n_components_, "com o PCA.  " \
      "Variâncias =", pca.explained_variance_ratio_)

print("5 fold:")
avg_knn_score = 0
for tr_5kf_idx, ts_5kf_idx in zip(tr_5kf_idxs, ts_5kf_idxs):
    # Define os subvetores x e y, para treino e teste (5-kfold)
    x_5kf_tr, y_5kf_tr = x_reduced[tr_5kf_idx], y[tr_5kf_idx] # Treino
    x_5kf_ts, y_5kf_ts = x_reduced[ts_5kf_idx], y[ts_5kf_idx] # Teste
    
    # Procura o melhor K com 3 folds (StratifiedKFold)
    gscv = GridSearchCV(
                KNeighborsClassifier(), 
                {'n_neighbors':ks}
           ).fit(x_5kf_tr, y_5kf_tr)
    
    # Guarda a acurácia do kNN para o melhor k do fold atual
    knn_score = gscv.score(x_5kf_ts, y_5kf_ts)
    print('    K = {:d}; Acurácia = {:.12f}'.format(
                gscv.best_params_["n_neighbors"], 
                knn_score)
         )
    avg_knn_score += knn_score
    
avg_knn_score /= 5
print("Acurácia média do kNN: {:.3f}".format(avg_knn_score))

Teste do kNN
10 dimensões reduzidas para 3 com o PCA.  Variâncias = [0.69274183 0.15198329 0.09115099]
5 fold:


/home/tales/virtualenvs/ML/lib/python3.6/site-packages/sklearn/model_selection/_split.py:2053: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)


    K = 21; Acurácia = 0.880382775120


/home/tales/virtualenvs/ML/lib/python3.6/site-packages/sklearn/model_selection/_split.py:2053: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)


    K = 25; Acurácia = 0.882775119617


...

    K = 21; Acurácia = 0.882634730539


/home/tales/virtualenvs/ML/lib/python3.6/site-packages/sklearn/model_selection/_split.py:2053: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)


    K = 21; Acurácia = 0.881437125749


/home/tales/virtualenvs/ML/lib/python3.6/site-packages/sklearn/model_selection/_split.py:2053: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)


    K = 21; Acurácia = 0.882634730539
Acurácia média do kNN: 0.882


Para o **SVM RBF** teste para $C = 2^{-5}, 2^{0}, 2^{5}\: e\: 2^{10}$ e $\gamma = 2^{-15}, 2^{-10}, 2^{-5}, 2^{0}\: e\: 2^{5}$.

In [4]:
# Conjunto de hiperparâmetros testados para otimização do SVM
Cs = [2**-5, 2**1, 2**5, 2**10]
gammas = [2**-15, 2**-10, 2**0, 2**5]

print("Teste do SVM")

print("5 fold:")
avg_svm_score = 0
for tr_5kf_idx, ts_5kf_idx in zip(tr_5kf_idxs, ts_5kf_idxs):
    # Define os subvetores x e y, para treino e teste (5-kfold)
    x_5kf_tr, y_5kf_tr = x[tr_5kf_idx], y[tr_5kf_idx] # Treino
    x_5kf_ts, y_5kf_ts = x[ts_5kf_idx], y[ts_5kf_idx] # Teste
    
    # Procura o melhor C e gamma com 3 folds (kernel='rbf' por padrão)
    gscv = GridSearchCV(
                SVC(), 
                {'C':Cs, 'gamma':gammas}
           ).fit(x_5kf_tr, y_5kf_tr)
    
    # Guarda a acurácia do SVC para o melhor C e gamma do fold atual
    svm_score = gscv.score(x_5kf_ts, y_5kf_ts)
    C = int(math.log(gscv.best_params_["C"], 2))
    gamma = int(math.log(gscv.best_params_["gamma"], 2))
    print('    C = 2^{:d}, gamma = 2^{:d}; Acurácia = {:.12f}'.format(
                C, 
                gamma, 
                svm_score)
         )
    avg_svm_score += svm_score
    
avg_svm_score /= 5
print("Acurácia média do SVM: {:.3f}".format(avg_svm_score))

Teste do SVM
5 fold:


/home/tales/virtualenvs/ML/lib/python3.6/site-packages/sklearn/model_selection/_split.py:2053: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)


    C = 2^10, gamma = 2^-10; Acurácia = 0.898325358852


/home/tales/virtualenvs/ML/lib/python3.6/site-packages/sklearn/model_selection/_split.py:2053: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)


    C = 2^10, gamma = 2^-10; Acurácia = 0.885167464115


...

    C = 2^10, gamma = 2^-10; Acurácia = 0.898203592814


/home/tales/virtualenvs/ML/lib/python3.6/site-packages/sklearn/model_selection/_split.py:2053: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)


    C = 2^10, gamma = 2^-10; Acurácia = 0.893413173653


/home/tales/virtualenvs/ML/lib/python3.6/site-packages/sklearn/model_selection/_split.py:2053: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)


    C = 2^10, gamma = 2^-10; Acurácia = 0.885029940120
Acurácia média do SVM: 0.892


Para a **rede neural**, teste com $3, 7, 10,\: e\: 20$ neurônios na camada escondida.

In [5]:
import warnings
warnings.filterwarnings("ignore")

# Conjunto de hiperparâmetros testados para otimização da rede neural
Ns = [(3,), (7,), (10,), (20,)]

print("Teste do RN")

print("5 fold:")
avg_rn_score = 0
for tr_5kf_idx, ts_5kf_idx in zip(tr_5kf_idxs, ts_5kf_idxs):
    # Define os subvetores x e y, para treino e teste (5-kfold)
    x_5kf_tr, y_5kf_tr = x[tr_5kf_idx], y[tr_5kf_idx] # Treino
    x_5kf_ts, y_5kf_ts = x[ts_5kf_idx], y[ts_5kf_idx] # Teste
    
    # Procura o melhor número de neurônios folds (StratifiedKFold)
    gscv = GridSearchCV(
                MLPClassifier(), 
                {'hidden_layer_sizes':Ns}
           ).fit(x_5kf_tr, y_5kf_tr)
    
    # Guarda a acurácia da Rede Neural para o melhor # neurônios do fold atual
    rn_score = gscv.score(x_5kf_ts, y_5kf_ts)
    print('    #Neurônios = {:2d}; Acurácia = {:.12f}'.format(
                gscv.best_params_["hidden_layer_sizes"][0], 
                rn_score)
         )
    avg_rn_score += rn_score
    
avg_rn_score /= 5
print("Acurácia média da Rede neural: {:.3f}".format(avg_rn_score))

Teste do RN
5 fold:
    #Neurônios =  7; Acurácia = 0.887559808612
    #Neurônios =  7; Acurácia = 0.900717703349
    #Neurônios =  3; Acurácia = 0.899401197605
    #Neurônios = 10; Acurácia = 0.900598802395
    #Neurônios =  7; Acurácia = 0.893413173653
Acurácia média da Rede neural: 0.896


Para o **RF**, teste $\texttt{maxfeatures = 2, 3, 5, 7}\; e\; \texttt{nestimators = 100, 200, 400 e 800}$.

In [6]:
# Conjunto de hiperparâmetros testados para otimização do RF
max_features = [2, 3, 5, 7]
n_estimators = [100, 200, 400, 800]

print("Teste do RF")

print("5 fold:")
avg_rf_score = 0
for tr_5kf_idx, ts_5kf_idx in zip(tr_5kf_idxs, ts_5kf_idxs):
    # Define os subvetores x e y, para treino e teste (5-kfold)
    x_5kf_tr, y_5kf_tr = x[tr_5kf_idx], y[tr_5kf_idx] # Treino
    x_5kf_ts, y_5kf_ts = x[ts_5kf_idx], y[ts_5kf_idx] # Teste
    
    # Procura os melhores hiperparâmetros 3 folds (StratifiedKFold)
    gscv = GridSearchCV(
                RandomForestClassifier(), 
                {'n_estimators':n_estimators, 'max_features':max_features}
           ).fit(x_5kf_tr, y_5kf_tr)
    
    # Guarda a acurácia da Random Forest para os hiperparâmetros do fold atual
    rf_score = gscv.score(x_5kf_ts, y_5kf_ts)
    print('    n_estimators = {:d}, max_features = {:d}; Acurácia = {:.12f}'.format(
                gscv.best_params_["n_estimators"], 
                gscv.best_params_["max_features"],
                rf_score)
         )
    avg_rf_score += rf_score
    
avg_rf_score /= 5
print("Acurácia média da Random Forest: {:.3f}".format(avg_rf_score))

Teste do RF
5 fold:
    n_estimators = 100, max_features = 2; Acurácia = 0.883971291866
    n_estimators = 100, max_features = 3; Acurácia = 0.895933014354
    n_estimators = 200, max_features = 2; Acurácia = 0.901796407186
    n_estimators = 100, max_features = 5; Acurácia = 0.882634730539
    n_estimators = 400, max_features = 3; Acurácia = 0.891017964072
Acurácia média da Random Forest: 0.891


Para o **GBM** teste para $\texttt{nÁrvores = 30, 70  e  100}$, com $\texttt{learningRate = 0.1 e 0.05}$, e $\texttt{profundidade da árvore = 5}$.

In [7]:
# Conjunto de hiperparâmetros testados para otimização do GBM
n_estimators = [30, 70, 100]
learning_rates = [0.1, 0.05]
max_depth = [5]

print("Teste do GBM")

print("5 fold:")
avg_gbm_score = 0
for tr_5kf_idx, ts_5kf_idx in zip(tr_5kf_idxs, ts_5kf_idxs):
    # Define os subvetores x e y, para treino e teste (5-kfold)
    x_5kf_tr, y_5kf_tr = x[tr_5kf_idx], y[tr_5kf_idx] # Treino
    x_5kf_ts, y_5kf_ts = x[ts_5kf_idx], y[ts_5kf_idx] # Teste
    
    # Procura os melhores hiperparâmetros 3 folds (StratifiedKFold)
    gscv = GridSearchCV(
                GradientBoostingClassifier(), {
                    'n_estimators':n_estimators, 
                    'learning_rate':learning_rates, 
                    'max_depth':max_depth}
           ).fit(x_5kf_tr, y_5kf_tr)
    
    # Guarda a acurácia da Random Forest para os hiperparâmetros do fold atual
    gbm_score = gscv.score(x_5kf_ts, y_5kf_ts)
    print('    n_estimators = {:3d}, learning_rate = {:.2f}; Acurácia = {:.12f}'.format(
                gscv.best_params_["n_estimators"], 
                gscv.best_params_["learning_rate"], 
                gbm_score)
         )
    avg_gbm_score += gbm_score
    
avg_gbm_score /= 5
print("Acurácia média da BGM: {:.3f}".format(avg_gbm_score))

Teste do GBM
5 fold:
    n_estimators =  70, learning_rate = 0.10; Acurácia = 0.873205741627
    n_estimators =  30, learning_rate = 0.10; Acurácia = 0.887559808612
    n_estimators = 100, learning_rate = 0.05; Acurácia = 0.900598802395
    n_estimators = 100, learning_rate = 0.10; Acurácia = 0.883832335329
    n_estimators =  70, learning_rate = 0.05; Acurácia = 0.887425149701
Acurácia média da BGM: 0.887


Para o algoritmo com maior acurácia, reporte o valor dos hiperparâmetros obtidos para gerar o classificador final.

In [8]:
print("O melhor classificador foi a Rede Neural com {:.3f} de acurácia média.".format(avg_rn_score))
gscv = GridSearchCV(
            MLPClassifier(), 
            {'hidden_layer_sizes':Ns}, 
            cv=5
       ).fit(x, y)
print('O melhor número de neurônios dentre os disponíveis é {:2d}' \
          .format(gscv.best_params_["hidden_layer_sizes"][0]))
print(gscv.best_estimator_)

O melhor classificador foi a Rede Neural com 0.896 de acurácia média.
O melhor número de neurônios dentre os disponíveis é 10
MLPClassifier(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(10,), learning_rate='constant',
       learning_rate_init=0.001, max_iter=200, momentum=0.9,
       n_iter_no_change=10, nesterovs_momentum=True, power_t=0.5,
       random_state=None, shuffle=True, solver='adam', tol=0.0001,
       validation_fraction=0.1, verbose=False, warm_start=False)


Vale ressaltar que, com exceção do **kNN**, os outros métodos tiveram um resultado somente 0.004 pior, mas com alguns o tempo de execução é consideravelmente mais rápido que a **rede neural**.